In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process

#https://www.kaggle.com/code/heeraldedhia/movie-ratings-and-recommendation-using-knn

c:\Users\jsbreite\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [1]:
# To store the data
import pandas as pd

# To do linear algebra
import numpy as np

# To create plots
import matplotlib.pyplot as plt
import seaborn as sns

# To create interactive plots
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# To shift lists
from collections import deque

# To compute similarities between vectors
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# To use recommender systems
import surprise as sp
from surprise.model_selection import cross_validate

# To create deep learning models
from keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout
from keras.models import Model

# To create sparse matrices
from scipy.sparse import coo_matrix

# To light fm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

# To stack sparse matrices
from scipy.sparse import vstack

c:\Users\jsbreite\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning:

LightFM was compiled without OpenMP support. Only a single thread will be used.



In [2]:
## Wenn Colabs benutzt wird, kann es damit verbunden werden

movie_tile_File = '/content/drive/MyDrive/Programmieprojekt/movie_titles.csv'
movie_tile_File_new = '/content/drive/MyDrive/Programmieprojekt/movie_titles_new.csv'
combined_data_1 = '/content/drive/MyDrive/Programmieprojekt/combined_data_1.txt'
combined_data_2 = '/content/drive/MyDrive/Programmieprojekt/combined_data_2.txt'
combined_data_3 = '/content/drive/MyDrive/Programmieprojekt/combined_data_3.txt'
combined_data_4 = '/content/drive/MyDrive/Programmieprojekt/combined_data_4.txt'
new_Combined = '/content/drive/MyDrive/Programmieprojekt/Cobined_data_new_v1.csv'
netflix_rating_Combined = '/content/drive/MyDrive/Programmieprojekt/netflix_data.csv'
parquet_combined_data = '/content/drive/MyDrive/Programmieprojekt/data_Comb.zip'

In [3]:
#Als Nächstes sollen alle Pfade mit variablen belegt werden, dass macht das austauschen einfacher.

movie_tile_File = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/movie_titles.csv'
movie_tile_File_new = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/movie_titles_new.csv'
combined_data_1 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_1.txt'
combined_data_2 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_2.txt'
combined_data_3 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_3.txt'
combined_data_4 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_4.txt'
new_Combined = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/Cobined_data_new_v1.csv'
netflix_rating_Combined = 'C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/netflix_data.csv'
parquet_combined_data = 'C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/data_Comb.zip'

In [4]:
#Load Data into viables

combined_data_1_raw = pd.read_csv(combined_data_1, header=None, names=['Cust_Id', 'Rating', 'Date'], usecols=[0, 1, 2])
combined_data_2_raw = pd.read_csv(combined_data_2, header=None, names=['Cust_Id', 'Rating', 'Date'], usecols=[0, 1, 2])
combined_data_3_raw = pd.read_csv(combined_data_3, header=None, names=['Cust_Id', 'Rating', 'Date'], usecols=[0, 1, 2])
combined_data_4_raw = pd.read_csv(combined_data_4, header=None, names=['Cust_Id', 'Rating', 'Date'], usecols=[0, 1, 2])

In [5]:
## Combined_DATA_1
# Find empty rows to slice dataframe for each movie
tmp_movies = combined_data_1_raw[combined_data_1_raw['Rating'].isna()]['Cust_Id'].reset_index()  
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

# Shift the movie_indices by one to get start and endpoints of all movies
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)


# Gather all dataframes
user_data = []

# Iterate over all movies
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    # Check if it is the last movie in the file
    if df_id_1<df_id_2:
        tmp_df = combined_data_1_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = combined_data_1_raw.loc[df_id_1+1:].copy()
        
    # Create movie_id column
    tmp_df['Movie_Id'] = movie_id
    
    # Append dataframe to list
    user_data.append(tmp_df)

# Combine all dataframes
df_1 = pd.concat(user_data)
del user_data, combined_data_1_raw, combined_data_1, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
# print('Shape User-Ratings:\t{}'.format(df_1.shape))

## Combined_DATA_2
# Find empty rows to slice dataframe for each movie
tmp_movies = combined_data_2_raw[combined_data_2_raw['Rating'].isna()]['Cust_Id'].reset_index()  ##---> Nachvollziehen
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

# Shift the movie_indices by one to get start and endpoints of all movies
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)


# Gather all dataframes
user_data = []

# Iterate over all movies
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    # Check if it is the last movie in the file
    if df_id_1<df_id_2:
        tmp_df = combined_data_2_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = combined_data_2_raw.loc[df_id_1+1:].copy()
        
    # Create movie_id column
    tmp_df['Movie_Id'] = movie_id
    
    # Append dataframe to list
    user_data.append(tmp_df)

# Combine all dataframes
df_2 = pd.concat(user_data)
del user_data, combined_data_2_raw, combined_data_2, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
# print('Shape User-Ratings:\t{}'.format(df_2.shape))

## Combined_DATA_3
# Find empty rows to slice dataframe for each movie
tmp_movies = combined_data_3_raw[combined_data_3_raw['Rating'].isna()]['Cust_Id'].reset_index()  ##---> Nachvollziehen
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

# Shift the movie_indices by one to get start and endpoints of all movies
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)


# Gather all dataframes
user_data = []

# Iterate over all movies
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    # Check if it is the last movie in the file
    if df_id_1<df_id_2:
        tmp_df = combined_data_3_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = combined_data_3_raw.loc[df_id_1+1:].copy()
        
    # Create movie_id column
    tmp_df['Movie_Id'] = movie_id
    
    # Append dataframe to list
    user_data.append(tmp_df)

# Combine all dataframes
df_3 = pd.concat(user_data)
del user_data, combined_data_3_raw, combined_data_3, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
# print('Shape User-Ratings:\t{}'.format(df_3.shape))

## Combined_DATA_4
# Find empty rows to slice dataframe for each movie
tmp_movies = combined_data_4_raw[combined_data_4_raw['Rating'].isna()]['Cust_Id'].reset_index()  ##---> Nachvollziehen
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

# Shift the movie_indices by one to get start and endpoints of all movies
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)


# Gather all dataframes
user_data = []

# Iterate over all movies
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    # Check if it is the last movie in the file
    if df_id_1<df_id_2:
        tmp_df = combined_data_4_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = combined_data_4_raw.loc[df_id_1+1:].copy()
        
    # Create movie_id column
    tmp_df['Movie_Id'] = movie_id
    
    # Append dataframe to list
    user_data.append(tmp_df)

# Combine all dataframes
df_4 = pd.concat(user_data)
del user_data, combined_data_4_raw, combined_data_4, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
print('Shape User-Ratings:\t{}'.format(df_4.shape))

#Zusammenfügen der aller Daten in einer Variable
data = [df_1, df_2,df_3,df_4]
df = pd.concat(data)
del df_1, df_2, df_3, df_4
print(df.tail(5))
print(df.head(5))

Shape User-Ratings:	(26847523, 4)
          Cust_Id  Rating        Date  Movie_Id
26851921  1790158     4.0  2005-11-01     17770
26851922  1608708     3.0  2005-07-19     17770
26851923   234275     1.0  2004-08-07     17770
26851924   255278     4.0  2004-05-28     17770
26851925   453585     2.0  2005-03-10     17770
   Cust_Id  Rating        Date  Movie_Id
1  1488844     3.0  2005-09-06         1
2   822109     5.0  2005-05-13         1
3   885013     4.0  2005-10-19         1
4    30878     4.0  2005-12-26         1
5   823519     3.0  2004-05-03         1


In [ ]:
print(movie_titles.head(5))

In [ ]:
test= df

In [ ]:
test = test.reset_index(level=0)

In [ ]:
print(test.head(5))

In [6]:
# Load data for all movies
from multiprocessing import dummy


movie_titles = pd.read_csv(movie_tile_File, 
                           encoding = 'ISO-8859-1', 
                           engine = 'python',
                           delimiter =',',
                           header = None, 
                           on_bad_lines= 'skip', #Wird gebraucht um Fehlerhafte Title zu behen.
                           
                           names = ['Id', 'Year', 'Name'])
                           

print('Shape Movie-Titles:\t{}'.format(movie_titles.shape))

#Zwischenspeichern der Movies, um fehlerhafte Liste zu bearbeiten
movie_titles.to_csv(movie_tile_File_new,encoding = 'ISO-8859-1',sep=';')

movie_titles.rename(columns={'Id':'Movie_Id'}, inplace=True)
display(movie_titles)

Shape Movie-Titles:	(17434, 3)


,Movie_Id,Year,Name
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
...,...,...,...
17429,17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17430,17767,2004.0,Fidel Castro: American Experience
17431,17768,2000.0,Epoch
17432,17769,2003.0,The Company


In [ ]:
print(movie_titles.index)

In [7]:
# Filter sparse movies
min_movie_ratings = 1000
filter_movies = (df['Movie_Id'].value_counts()>min_movie_ratings)
filter_movies = filter_movies[filter_movies].index.tolist()

# Filter sparse users
min_user_ratings = 200
filter_users = (df['Cust_Id'].value_counts()>min_user_ratings)
filter_users = filter_users[filter_users].index.tolist()

# Actual filtering
df_filterd = df[(df['Movie_Id'].isin(filter_movies)) & (df['Cust_Id'].isin(filter_users))]
del filter_movies, filter_users, min_movie_ratings, min_user_ratings


print('Shape User-Ratings unfiltered:\t{}'.format(df.shape))
print('Shape User-Ratings filtered:\t{}'.format(df_filterd.shape))

Shape User-Ratings unfiltered:	(100480507, 4)
Shape User-Ratings filtered:	(75010415, 4)


In [ ]:
Movie_Gerne = pd.read_table('C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/IDMB/title.basics.tsv')
genres = Movie_Gerne.drop(['tconst', 'titleType', 'isAdult','startYear', 'endYear', 'runtimeMinutes', 'primaryTitle'],axis =1)
del Movie_Gerne
genres['genres'] = genres['genres'].str.split(',').str[0]

In [ ]:
print('movie_titles Bevor Duplicates\t{}'.format(movie_titles.shape))

movie_titles = movie_titles.drop_duplicates(subset=['Year', 'Name'],keep='first')
print('movie_titles Nach entfernen Duplicates\t{}'.format(movie_titles.shape))

data = pd.merge(pd.DataFrame(movie_titles), pd.DataFrame(genres), left_on=['Name'], 
             right_on= ['originalTitle'], how='left')
print(data.head(5))
print(movie_titles.head(5))
print('Data\t{}'.format(data.shape))
print('movie_titles\t{}'.format(movie_titles.shape))
##aa
data = data.drop_duplicates(subset=['Name'],keep='first')
print('Data\t{}'.format(data.shape))
print('movie_titles\t{}'.format(movie_titles.shape))

In [ ]:
df = data = pd.merge(pd.DataFrame(data), pd.DataFrame(df), left_on=['Id'], 
             right_on= ['Id'], how='inner')

In [ ]:
plt.subplots(figsize=(12,10))
list1 = []
for i in data['genres']:
    list1.extend(i)
ax = pd.Series(list1).value_counts()[:10].sort_values(ascending=True).plot.barh(width=0.9,color=sns.color_palette('hls',10))
for i, v in enumerate(pd.Series(list1).value_counts()[:10].sort_values(ascending=True).values): 
    ax.text(.8, i, v,fontsize=12,color='white',weight='bold')
plt.title('Top Genres')
plt.show()

In [ ]:
#                                                                                                                                                               https://www.kaggle.com/code/heeraldedhia/movie-ratings-and-recommendation-using-knn

import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')
import base64
import io
from matplotlib.pyplot import imread
import codecs
from IPython.display import HTML

from surprise import Reader
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [ ]:
reader = Reader()
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']], reader)
data.split(n_folds=10)



In [152]:
df_filterd.tail(10)

,Cust_Id,Rating,Date,Movie_Id
26850990,234275,1.0,2004-10-04,17769
26850991,2198837,2.0,2004-10-20,17769
26850992,359157,3.0,2005-12-30,17769
26850993,391432,4.0,2004-12-31,17769
26850995,1428223,4.0,2005-02-01,17769
26850996,439703,2.0,2005-02-06,17769
26850998,483107,4.0,2005-04-08,17769
26851000,77664,2.0,2005-09-16,17769
26851001,1922916,3.0,2005-10-19,17769
26851003,1704416,2.0,2005-12-16,17769


In [16]:
                                                                                                                                                                ### https://www.kaggle.com/code/ecemboluk/recommendation-system-with-cf-using-knn

data = pd.merge(movie_titles,df_filterd)
data = data.iloc[:26851003,:]
user_movie_table = data.pivot_table(index = ["Movie_Id"],columns = ["Cust_Id"],values = "Rating").fillna(0)

user_movie_table.head(10)


Cust_Id,10,1000033,1000062,1000079,1000084,1000095,1000104,1000170,1000192,1000195,...,999768,999836,999844,999892,999901,999903,999913,99993,999944,999964
Movie_Id,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [20]:
query_index = np.random.choice(user_movie_table.shape[0])
print("Choosen Movie is: ",user_movie_table.index[query_index])

Choosen Movie is:  5625


In [11]:
user_movie_table.tail(1000000)

Cust_Id,10,1000033,1000062,1000079,1000084,1000095,1000104,1000170,1000192,1000195,...,999768,999836,999844,999892,999901,999903,999913,99993,999944,999964
Movie_Id,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6464,2.0,0.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,4.0,...,0.0,3.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0
6470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
user_movie_table_matrix = csr_matrix(user_movie_table.values)
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(user_movie_table_matrix)
distances, indices = model_knn.kneighbors(user_movie_table.iloc[query_index,:].values.reshape(1,-1), n_neighbors = 38)

In [21]:
movie = []
distance = []

for i in range(0, len(distances.flatten())):
    if i != 0:
        movie.append(user_movie_table.index[indices.flatten()[i]])
        distance.append(distances.flatten()[i])    

m=pd.Series(movie,name='movie')
d=pd.Series(distance,name='distance')
recommend = pd.concat([m,d], axis=1)
recommend = recommend.sort_values('distance',ascending=False)

print('Recommendations for {0}:\n'.format(user_movie_table.index[query_index]))
for i in range(0,recommend.shape[0]):
    print('{0}: {1}, with distance of {2}'.format(i, recommend["movie"].iloc[i], recommend["distance"].iloc[i]))

Recommendations for 5625:

0: 1307, with distance of 0.4672247591429809
1: 5814, with distance of 0.4664577911386315
2: 457, with distance of 0.4661795600507668
3: 5571, with distance of 0.46214525885404045
4: 6206, with distance of 0.4621344308025198
5: 798, with distance of 0.46188677230847097
6: 4640, with distance of 0.46073218196076093
7: 4656, with distance of 0.45869920129191466
8: 3962, with distance of 0.4572285666121797
9: 2112, with distance of 0.4562361356477692
10: 3825, with distance of 0.45544743559746215
11: 6196, with distance of 0.4540098632851287
12: 5862, with distance of 0.4526918696533875
13: 1798, with distance of 0.4519131372291323
14: 2372, with distance of 0.45082093630959674
15: 607, with distance of 0.4508197600163456
16: 708, with distance of 0.4492622385672983
17: 5054, with distance of 0.44925930309211815
18: 5732, with distance of 0.44742512239673715
19: 3925, with distance of 0.446195501690397
20: 3860, with distance of 0.44147686590366964
21: 1220, wit